## Camera

In [1]:
from jetcam.csi_camera import CSICamera
# from jetcam.usb_camera import USBCamera

camera = CSICamera(width=224, height=224)
# camera = USBCamera(width=224, height=224)

camera.running = True

In [1]:
from jetracer.nvidia_racecar import NvidiaRacecar
from jetcam.csi_camera import CSICamera

In [2]:
STEERING_GAIN = 1
STEERING_BIAS = 0

class JetRacer(object):
    def __init__(self, camera_width=112, camera_height=112, fps=30):
        self.car = NvidiaRacecar()
        print("====== LOADING CAMERA ======")
        self.camera = CSICamera(width=camera_width, height=camera_height, capture_fps=fps)
        self.camera.running = True
        print("====== CAMERA LOADED SUCCESSFULLY ======")
        self.car.throttle = 0
        self.car.steering = 0

    def apply_throttle(self, throttle):
        self.car.throttle = float(throttle)

    def apply_steering(self, steer):
        self.car.steering = float(steer) * STEERING_GAIN + STEERING_BIAS

    def get_image(self):
        return self.camera.value.copy()

    def resize_image(self, image, width, height):
        return cv2.resize(image, (width, height))

In [2]:
import os
import subprocess

import cv2
import torch
import numpy as np
import pandas as pd

class XYDataset(torch.utils.data.Dataset):
    def __init__(self, directory, transform=None, random_hflip=False):
        super(XYDataset, self).__init__()
        self.directory = directory
        self.transform = transform
        self.annotations = []
        self.random_hflip = random_hflip
        self.n_items = 0
        
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self, idx):
        annot = self.annotations[idx]
        steer, throttle = annot['steer'], annot['throttle']
        image = cv2.imread(annot['image_path'], cv2.IMREAD_COLOR)
        image = PIL.Image.fromarray(image)
        width = image.width
        height = image.height
        if self.transform is not None:
            image = self.transform(image)
        
        if self.random_hflip and float(np.random.random(1)) > 0.5:
            image = torch.from_numpy(image.numpy()[..., ::-1].copy())
            
        return image, steer, throttle
        
    def add_annotation(self, image_path, steer, throttle):
        self.annotations += [{
            'image_path': image_path,
            'steer': steer,
            'throttle': throttle
        }]
        
    def save_entry(self, image, steer, throttle):
        if not os.path.exists(self.directory):
            subprocess.call(['mkdir', '-p', self.directory])
        
        filename = f"{self.n_items}.jpg"
        self.n_items += 1

        # Save image
        image_path = os.path.join(self.directory, filename)
        cv2.imwrite(image_path, image)
        
        # Save control commands
        self.add_annotation(image_path, steer, throttle)
        pd.DataFrame(self.annotations).to_csv(os.path.join(self.directory, "annot"), index=False)
        
#     def get_count(self, category):
#         i = 0
#         for a in self.annotations:
#             if a['category'] == category:
#                 i += 1
#         return i

In [4]:
import ipywidgets.widgets as widgets

car = JetRacer()
controller = widgets.Controller(index=0)

TASK = 'data'
dataset = XYDataset(TASK, None, False)

====== LOADING CAMERA ======


RuntimeError: Could not initialize camera.  Please see error trace.

In [5]:
car.apply_steering(controller.axes[0].value)
car.apply_throttle(controller.axes[3].value)
#     dataset.save_entry(car.get_image(), controller.axes[0], controller.axes[3])

KeyboardInterrupt: 

## Controller

In [2]:
from jetracer.nvidia_racecar import NvidiaRacecar
import traitlets

car = NvidiaRacecar()

In [3]:
car.throttle_gain = 1.0
car.steering_offset = 0

# # Fix the throttle
# car.steering = 0
# car.throttle = -0.4

In [4]:
import ipywidgets.widgets as widgets

controller = widgets.Controller(index=0)

In [5]:
left_link = traitlets.dlink((controller.axes[0], 'value'), (car, 'steering'), transform=lambda x: -x)
right_link = traitlets.dlink((controller.axes[3], 'value'), (car, 'throttle'), transform=lambda x: 0.4 * x)

## Task

In [38]:
import os
import subprocess

import cv2
import torch
import numpy as np
import pandas as pd

class XYDataset(torch.utils.data.Dataset):
    def __init__(self, directory, transform=None, random_hflip=False):
        super(XYDataset, self).__init__()
        self.directory = directory
        self.transform = transform
        self.annotations = []
        self.random_hflip = random_hflip
        self.n_items = 0
        
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self, idx):
        annot = self.annotations[idx]
        steer, throttle = annot['steer'], annot['throttle']
        image = cv2.imread(annot['image_path'], cv2.IMREAD_COLOR)
        image = PIL.Image.fromarray(image)
        width = image.width
        height = image.height
        if self.transform is not None:
            image = self.transform(image)
        
        if self.random_hflip and float(np.random.random(1)) > 0.5:
            image = torch.from_numpy(image.numpy()[..., ::-1].copy())
            
        return image, steer, throttle
        
    def add_annotation(self, image_path, steer, throttle):
        self.annotations += [{
            'image_path': image_path,
            'steer': steer,
            'throttle': throttle
        }]
        
    def save_entry(self, image, steer, throttle):
        if not os.path.exists(self.directory):
            subprocess.call(['mkdir', '-p', self.directory])
        
        filename = f"{self.n_items}.jpg"
        self.n_items += 1

        # Save image
        image_path = os.path.join(self.directory, filename)
        cv2.imwrite(image_path, image)
        
        # Save control commands
        self.add_annotation(image_path, steer, throttle)
        pd.DataFrame(self.annotations).to_csv(os.path.join(self.directory, "annot"), index=False)
        
#     def get_count(self, category):
#         i = 0
#         for a in self.annotations:
#             if a['category'] == category:
#                 i += 1
#         return i

In [39]:
import torchvision.transforms as transforms

TASK = 'data'

# TRANSFORMS = transforms.Compose([
#     transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
#     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
# ])

dataset = XYDataset(TASK, None, False)

## Data Collection

In [9]:
SAVE_FREQ = 10

# while True:
#     START_DATA_COLLECTION = controller.buttons[4].value
#     STOP_DATA_COLLECTION = controller.buttons[0].value
#     i = 0
#     if START_DATA_COLLECTION:
#         START_DATA_COLLECTION = 1
#         i += 1
#         if i % SAVE_FREQ == 0: 
#             # get the image
#             img = camera.value

#             # get steer
#             steer = left_link.source[0].value

#             # get throttle
#             throttle = right_link.source[0].value

#             dataset.save_entry(img, steer, throttle)

#         if STOP_DATA_COLLECTION:
#             break

# left_link = traitlets.dlink((controller.axes[0], 'value'), (car, 'steering'), transform=lambda x: -x)
# right_link = traitlets.dlink((controller.axes[3], 'value'), (car, 'throttle'), transform=lambda x: 0.4 * x)

while True:
    # get the image
#     img = camera.value

    # get steer
    steer = left_link.source[0].value

    # get throttle
    throttle = right_link.source[0].value

#     dataset.save_entry(img, steer, throttle)

KeyboardInterrupt: 